## IMPORT

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Constants
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 128
BATCH_SIZE = 228  # Adjust for available VRAM
NUM_CLASSES = 6  # Fresh/Rotten for each fruit
BRUISED_CLASSES = 2  # Bruised or not bruised
# full file path
# C:\Users\Kenan\Downloads\CNN_train_test_model\datasplit
DATASET_PATH = 'datasplit'  # Change to the dataset path
NUM_EPOCHS = 10

In [2]:

# Preprocessing transformations
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset and Dataloaders
train_dataset = ImageFolder(root=f"{DATASET_PATH}/train", transform=transform)
val_dataset = ImageFolder(root=f"{DATASET_PATH}/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [3]:

# Model definition
class MultiTaskClassifier(nn.Module):
    def __init__(self, base_model, num_classes, bruised_classes, is_efficientnet=False):
        super(MultiTaskClassifier, self).__init__()
        self.base_model = base_model
        
        # Check if the model is EfficientNet
        if is_efficientnet:
            # For EfficientNet, replace the classifier
            in_features = self.base_model.classifier[1].in_features
            self.base_model.classifier = nn.Identity()
        else:
            # For ResNet, replace the fc layer
            in_features = self.base_model.fc.in_features
            self.base_model.fc = nn.Identity()
        
        # Define new classification layers
        self.classifier = nn.Linear(in_features, num_classes)
        self.bruised_classifier = nn.Linear(in_features, bruised_classes)

    def forward(self, x):
        x = self.base_model(x)  # Feature extraction
        fruit_class = self.classifier(x)  # Fruit and freshness classification
        bruised_class = self.bruised_classifier(x)  # Bruised/Not Bruised classification
        return fruit_class, bruised_class


In [4]:

# Function to train a model
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_preds = 0
        total_samples = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            
            # Zero the gradients
            optimizer.zero_grad()
            
            # Forward pass
            fruit_pred, bruised_pred = model(inputs)
            loss1 = criterion(fruit_pred, labels)  # Fruit type and freshness loss
            loss2 = criterion(bruised_pred, labels % 2)  # Bruised/Not Bruised loss
            loss = loss1 + loss2
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item()
            _, preds = torch.max(fruit_pred, 1)
            correct_preds += (preds == labels).sum().item()
            total_samples += labels.size(0)
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct_preds / total_samples
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")
    
    print("Training complete!")
    return model



In [5]:

# Function to evaluate a model
def evaluate_model(model, val_loader):
    model.eval()
    all_preds, all_labels = [], []
    all_bruised_preds, all_bruised_labels = [], []
    val_loss = 0.0

    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            
            # Forward pass
            fruit_pred, bruised_pred = model(inputs)
            loss1 = criterion(fruit_pred, labels)
            loss2 = criterion(bruised_pred, labels % 2)
            loss = loss1 + loss2
            val_loss += loss.item()

            # Store predictions
            all_preds.extend(torch.argmax(fruit_pred, dim=1).cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_bruised_preds.extend(torch.argmax(bruised_pred, dim=1).cpu().numpy())
            all_bruised_labels.extend((labels % 2).cpu().numpy())

    # Metrics
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    accuracy = accuracy_score(all_labels, all_preds)
    avg_loss = val_loss / len(val_loader)

    return precision, recall, f1, accuracy, avg_loss



In [6]:

# Train and evaluate ResNet
print("Training ResNet...")
resnet_model = MultiTaskClassifier(models.resnet50(weights=models.ResNet50_Weights.DEFAULT), NUM_CLASSES, BRUISED_CLASSES, is_efficientnet=False).to(DEVICE)
optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)
resnet_model = train_model(resnet_model, train_loader, nn.CrossEntropyLoss(), optimizer, NUM_EPOCHS)
resnet_metrics = evaluate_model(resnet_model, val_loader)
print(f"ResNet Metrics: Precision: {resnet_metrics[0]:.4f}, Recall: {resnet_metrics[1]:.4f}, F1-Score: {resnet_metrics[2]:.4f}, Accuracy: {resnet_metrics[3]:.4f}, Loss: {resnet_metrics[4]:.4f}")


Training ResNet...
Epoch 1/10, Loss: 2.3668, Accuracy: 0.2525
Epoch 2/10, Loss: 1.3990, Accuracy: 0.7150
Epoch 3/10, Loss: 0.6499, Accuracy: 0.8875
Epoch 4/10, Loss: 0.3315, Accuracy: 0.9625
Epoch 5/10, Loss: 0.2143, Accuracy: 0.9650
Epoch 6/10, Loss: 0.1181, Accuracy: 0.9850
Epoch 7/10, Loss: 0.0466, Accuracy: 0.9925
Epoch 8/10, Loss: 0.0187, Accuracy: 1.0000
Epoch 9/10, Loss: 0.0204, Accuracy: 1.0000
Epoch 10/10, Loss: 0.0047, Accuracy: 1.0000
Training complete!
ResNet Metrics: Precision: 0.4961, Recall: 0.5089, F1-Score: 0.4470, Accuracy: 0.5089, Loss: 7.5313


c:\Users\Kenan\anaconda3\envs\NvidiaEnv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:

# Train and evaluate EfficientNet
print("Training EfficientNet...")
efficientnet_model = MultiTaskClassifier(models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT),NUM_CLASSES, BRUISED_CLASSES, is_efficientnet=True).to(DEVICE)
optimizer = optim.Adam(efficientnet_model.parameters(), lr=0.001)
efficientnet_model = train_model(efficientnet_model, train_loader, nn.CrossEntropyLoss(), optimizer, NUM_EPOCHS)
efficientnet_metrics = evaluate_model(efficientnet_model, val_loader)
print(f"EfficientNet Metrics: Precision: {efficientnet_metrics[0]:.4f}, Recall: {efficientnet_metrics[1]:.4f}, F1-Score: {efficientnet_metrics[2]:.4f}, Accuracy: {efficientnet_metrics[3]:.4f}, Loss: {efficientnet_metrics[4]:.4f}")


Training EfficientNet...


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\Kenan/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:14<00:00, 1.50MB/s]


Epoch 1/10, Loss: 2.2804, Accuracy: 0.3550
Epoch 2/10, Loss: 1.3031, Accuracy: 0.7900
Epoch 3/10, Loss: 0.7590, Accuracy: 0.8875
Epoch 4/10, Loss: 0.4342, Accuracy: 0.9275
Epoch 5/10, Loss: 0.2172, Accuracy: 0.9775
Epoch 6/10, Loss: 0.1173, Accuracy: 0.9825
Epoch 7/10, Loss: 0.0636, Accuracy: 0.9975
Epoch 8/10, Loss: 0.0464, Accuracy: 0.9900
Epoch 9/10, Loss: 0.0794, Accuracy: 0.9900
Epoch 10/10, Loss: 0.0309, Accuracy: 0.9975
Training complete!
EfficientNet Metrics: Precision: 0.6240, Recall: 0.6071, F1-Score: 0.5856, Accuracy: 0.6071, Loss: 4.3647


In [8]:

# Save the models
torch.save(resnet_model.state_dict(), "resnet_fruit_model.pth")
torch.save(efficientnet_model.state_dict(), "efficientnet_fruit_model.pth")
print("Models saved!")


Models saved!
